In [3]:

#%matplotlib qt
#%matplotlib inline

#remember to do "conda activate mne" before launching the jupyter notebook
from functools import partial
from scipy import signal

import h5py
import multiprocessing as mp
import numpy as np
import pandas as pd
import logging
import os
#import pyeeg
#import mne
from functions import feature_frequency
from functions import feature_time


#-----------------------------------------------------------------------------
datasetPart = 1
prePath = "/Users/tinaraissi/workspace/EEG/tuh-eeg-auto-diagnosis/"
rootdir = {1: prePath+"v1.4.0_1/edf/train/02_tcp_le/", 2: prePath+"v1.4.0_2/edf/train/03_tcp_ar_a/"}

segLabelFilenames = {}


for subdir, dirs, files in os.walk(rootdir[datasetPart]):
    for file in files:
        p = os.path.join(subdir, file)
        if p.endswith("edf"):
            segLabelFilenames[p[56:-4]] = p.split(".edf")[0]
            
            
#------------------------------------------------------------------------------
#Wanted Channels

wanted_elecs = ['A1', 'A2', 'C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FP1',
                'FP2', 'FZ', 'O1', 'O2',
                'P3', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']

labels = {'bckg': 1 ,'fnsz': 2,'gnsz': 3,'spsz': 4,'cpsz': 5,'absz':6,'tnsz': 7,'tcsz': 8,'mysz': 9}


WINDOWS = [
    'barthann',
    'bartlett',
    'blackman',
    'blackmanharris',
    'bohman',
    'boxcar',
    'cosine',
    'flattop',
    'hamming',
    'hann',
    'nuttall',
    'parzen',
    'triang'
]

_bands = [1,4,8,12,18,24,30,60,90] 
time_threshold=100
start_time_shift=0.05
end_time_shift=0.05
power_line_frequency=60
low_cut=.2
high_cut=100

In [4]:
def remove_start_end_artifacts(segment):
    """ Removes self.start_time_shift percent of the recording from the beginning and self.end_time_shift from the
    end, since these parts often showed artifacts. """

    new_start_time_shift = int(start_time_shift * segment.duration)
    new_end_time_shift = int(end_time_shift * segment.duration)


    segment.signals = segment.signals[:, new_start_time_shift * segment.sampling_freq : -new_end_time_shift * segment.sampling_freq]
    segment.duration = segment.duration - (new_start_time_shift + new_end_time_shift)

    return segment

def filter_power_line_frequency(segment):
    """ Remove the power line frequency from the recordings """
    segment.signals = mne.filter.notch_filter(segment.signals,
                                              segment.sampling_freq,
                                              np.arange(power_line_frequency,
                                                        segment.sampling_freq/2,
                                                        power_line_frequency),
                                              verbose='error')
    return segment
def bandpass_time_domain(segment):
    """ filters the signal to frequency range self.low_cut - self.high_cut """
    segment.signals = mne.filter.filter_data(segment.signals,
                                             segment.sampling_freq,
                                             low_cut,
                                             high_cut,
                                             verbose='error')
    return segment
def volts_to_microvolts(segment):
    segment.signals *= 1000000
    return segment

def clean(segment):
    #segment = remove_start_end_artifacts(segment)
    segment = filter_power_line_frequency(segment)
    # TODO: this seems to "recenter" the data! find out why and how
    segment = bandpass_time_domain(segment)

    # drastically reduce amount of data by grabbing 1 minute of recording from the middle
    # rec = self.cut_one_minute(rec)

    # transform signal amplitudes from volts to microvolts
    segment = volts_to_microvolts(segment)
    return segment

In [5]:
class DataSplitter(object):
    """
    """

    @staticmethod
# ______________________________________________________________________________________________________________________
    def get_supported_windows():
        return WINDOWS


# ______________________________________________________________________________________________________________________
    def windows_weighted(self, windows, window_size):
        """ weights the splitted signal by the specified window function
        :param windows: the signals splitted into time windows
        :param window_size: the number of samples in the window
        :return: the windows weighted by the specified window function
        """
        method_to_call = getattr(signal, self.window)
        window = method_to_call(window_size)

        return windows * window

# ______________________________________________________________________________________________________________________
    def split(self, signals, sampling_freq):
        """ written by robin schirrmeister, adapted by lukas gemein
        :param rec: the recording object holding the signals and all information needed
        :return: the signals split into time windows of the specified size
        """
        window_size = int(sampling_freq * self.window_size_sec)
        overlap_size = int(self.overlap * window_size)
        stride = window_size - overlap_size

        if stride == 0:
            logging.error("Time windows cannot have an overlap of 100%.")

        # written by robin tibor schirrmeister
        signal_crops = []
        for i_start in range(0, signals.shape[-1] - window_size + 1, stride):
            signal_crops.append(np.take(signals, range(i_start, i_start + window_size), axis=-1, ))

        return self.windows_weighted(np.array(signal_crops), window_size)

# ______________________________________________________________________________________________________________________
    def __init__(self, overlap=50, window='boxcar', window_size_sec=2):
        self.overlap = overlap/100
        self.window = window
        self.window_size_sec = window_size_sec



In [6]:
class Recording(object):
    """ This is a container class for all the relevant data of a EEG recording
    """

    def __init__(self, data_set, edf_file_path, raw_edf, sampling_freq, n_samples, n_signals, signal_names, duration,
                 label_info_list, signals=None, signals_complete=None, signals_ft=None):
        self.data_set = data_set
        self.edf_file_path = edf_file_path
        self.raw_edf = raw_edf
        self.sampling_freq = sampling_freq
        self.n_samples = n_samples
        self.n_signals = n_signals
        self.signal_names = signal_names
        self.duration = duration
        self.signals = signals
        self.signal_ft = signals_ft
        self.signals_complete = signals_complete
        self.label_info_list = label_info_list

        
    def init_processing_units(self):
        self.splitter = DataSplitter(overlap = 50, window_size_sec=2)
        #self.feature_generator = feature_generator.FeatureGenerator(domain=cmd_args.domain, bands=cmd_args.bands,
        #                                                            window_size_sec=2,
        #                                                            overlap=50,
        #                                                           electrodes=wanted_elecs)
        
        
class Segment(object):
    
    def __init__(self, sampling_freq, n_samples, signal_names, duration, label,
                 signals, signals_ft=None):

        self.sampling_freq = sampling_freq
        self.n_samples = n_samples
        self.signal_names = signal_names
        self.duration = duration
        self.signals = signals
        self.label= label
        self.signals_ft = signals_ft

In [15]:
def getWantedChannelsFromRecording(rec_channels, wanted_elecs):
    
    selected_ch_names = []
    for wanted_part in wanted_elecs:
        wanted_found_name = []
        for ch_name in rec_channels:
            if ' ' + wanted_part + '-' in ch_name:
                wanted_found_name.append(ch_name)
        print(wanted_found_name)
        assert len(wanted_found_name) == 1
        selected_ch_names.append(wanted_found_name[0])
    return selected_ch_names

In [8]:
def get_segmet_with_label_from_recording(rec):
    segments = []
    for l in rec.label_info_list:
        startTime = float(l[0])
        endTime = float(l[1])
        startIndex = int(startTime*rec.sampling_freq)
        endIndex = int(endTime*rec.sampling_freq)
        label = int(l[2])
        duration = round(endTime - startTime, 4)
        n_samples = duration * rec.sampling_freq
        signals = rec.signals[:,startIndex:endIndex]
        s = Segment(rec.sampling_freq, n_samples, rec.signal_names, duration, label, signals)
        segments.append(s)
    return segments
        
        
    

In [9]:
def getLabelAndTimeStartAndEnd(filename):
    returnList = []
    with open(filename) as file:
        for line in file:
            if len(line.split()) == 4:
                returnList.append(line.split()[:-1])
    return returnList

In [10]:
def get_recording_with_mne(file_path):
    """ read info from the edf file without loading the data. loading data is done in multiprocessing since it takes
    some time. getting info is done before because some files had corrupted headers or weird sampling frequencies
    that caused the multiprocessing workers to crash. therefore get and check e.g. sampling frequency and duration
    beforehand
    :param file_path: path of the recording file
    :return: file name, sampling frequency, number of samples, number of signals, signal names, duration of the rec
    """
    
    edf_file_path = file_path+".edf" 
    try:
        edf_file = mne.io.read_raw_edf(file_path+".edf", verbose='error')
        labelLists = getLabelAndTimeStartAndEnd(file_path+".tse")
    except ValueError:
        return None, None, None, None, None, None
        # fix_header(file_path)
        # try:
        #     edf_file = mne.io.read_raw_edf(file_path, verbose='error')
        #     logging.warning("Fixed it!")
        # except ValueError:
        #     return None, None, None, None, None, None

    # some recordings have a very weird sampling frequency. check twice before skipping the file
    sampling_frequency = int(edf_file.info['sfreq'])
    if sampling_frequency < 10:
        return None
        #sampling_frequency = 1 / (edf_file.times[1] - edf_file.times[0])
        #if sampling_frequency < 10:
        #    return None, sampling_frequency, None, None, None, None
    n_samples = edf_file.n_times
    signal_names = edf_file.ch_names
    n_signals = len(signal_names)
    # some weird sampling frequencies are at 1 hz or below, which results in division by zero
    duration = n_samples / max(sampling_frequency, 1)
    label_info_list = []
    for ele in labelLists:
        ele[2] = labels[ele[2]]
        label_info_list.append(ele)
    
     
    return edf_file_path, edf_file, sampling_frequency, n_samples, n_signals, signal_names, duration, label_info_list


In [19]:
def load_data_with_mne_for_electrodes_in_signal_names(rec):
    """ loads the data using the mne library
    :param rec: recording object holding all necessary data of an eeg recording
    :return: a pandas dataframe holding the data of all electrodes as specified in the rec object
    """
    rec.raw_edf.load_data()
    signals = rec.raw_edf.get_data()
    wantedChannels = getWantedChannelsFromRecording(rec.signal_names, wanted_elecs)

    data = pd.DataFrame(index=range(rec.n_samples), columns=wantedChannels)
    for electrode in wantedChannels:
        data[electrode] = signals[list(rec.signal_names).index(electrode)]


    return data.values.T

In [17]:
def get_all_segments():
    segmentDicTLabel = dict(zip(list(labels.values()),[[] for _ in range(len((labels.keys())))]))
    for index, fName in enumerate(segLabelFilenames.keys()):
        print("working on recording ", str(index) +"/"+ str(len(segLabelFilenames.keys())))
        rec = Recording("/", *get_recording_with_mne(segLabelFilenames[fName]))
        rec.signals = load_data_with_mne_for_electrodes_in_signal_names(rec)
        segs = get_segmet_with_label_from_recording(rec)
        for s in segs:
            segmentDicTLabel[s.label].append(clean(s))
            
    return segmentDicTLabel

        
    
    

In [13]:
def get_ft_from_samples(signal_samples):
    windows = self.splitter.split(rec)
    rec.signals = windows
    rec.signals_ft = np.fft.rfft(windows, axis=2)

    rec.signals_ft = np.abs(rec.signals_ft)

In [14]:
def write_hdf5(segmentList, label):
    """ writes features to hdf5 file
    :param features: a matrix holding a feature vector for every recordings (maybe someday for every time window of
    every recording)
    :param in_dir: input directory used to extract the name if the class
    :param cmd_args: used to include important information in the file name s.a. window, windowsize etc
    :return: the name of the feature file
    """
    for ind, s in enumerate(segmentList):
        file_name = "hdf"+str(datasetPart)+"/"+str(label)+"/" + str(ind) + ".hdf"
        if not os.path.exists(file_name):
            print("writing "+str(file_name))
            hdf5_f= h5py.File(file_name, 'w')
            dset = hdf5_f.create_dataset('data', s.signals.shape, data= s.signals)
            dset.attrs["sampling_freq"] = s.sampling_freq
            dset.attrs["channel_names"] = s.signal_names
            dset.attrs["duration"] = s.duration
            dset.attrs["label"] = s.label
            hdf5_f.close()
        
"""

write:

remember you read it in this way

with h5py.File("p.hdf", "r") as f:
    a = f["data"]
    print(a.attrs["sampling_freq"])
    X = a[:]
"""        

'\n\nwrite:\n\nremember you read it in this way\n\nwith h5py.File("p.hdf", "r") as f:\n    a = f["data"]\n    print(a.attrs["sampling_freq"])\n    X = a[:]\n'

In [20]:
allSegments = get_all_segments()

#for k in allSegments.keys():
#    write_hdf5(allSegments[k], str(k))

In [14]:
#experiment on one recording
#after this step I have a recording object with signal samples

#p = list(segLabelFilenames.keys())[30]
#edf_file = mne.io.read_raw_edf(segLabelFilenames[p]+".edf", verbose='error')
#edf_file.load_data()
#data = edf_file.get_data()



#p = list(segLabelFilenames.keys())[30]
#edfFile = get_info_with_mne(segLabelFilenames[p]+".edf")
#labelLists = getLabelAndTimeStartAndEnd(segLabelFilenames[p]+".tse")
#rec = Recording("/", *get_recording_with_mne(segLabelFilenames[p]))
#create hdf starting by recording, take the segment of the label, add samples for that segment, 
#and use other information, like sampling frequency, duration, sex, age, label, channels
#rec.signals = load_data_with_mne_for_electrodes_in_signal_names(rec) 
#cleanedRec = clean(rec)
#cleanedRec.init_processing_units()

In [115]:
def compute_freq_feats(freq_feat_name, segment, window_size_sec):
        """ Computes the feature values for a given recording. Computes the value the frequency bands as specified in
        band_limits. The values are mean over all time windows and channels
        :param freq_feat_name: the function name that should be called
        :param rec: the recording object holding the data and info
        :return: mean amplitudes in frequency bands over the different channels
        """
        func = getattr(feature_frequency, freq_feat_name)
        # amplitudes shape: windows x electrodes x frequencies
        amplitudes = np.abs(segment.signals_ft)
        window_size = window_size_sec * segment.sampling_freq
        freq_bin_size = segment.sampling_freq / window_size

        mean_amplitudes = []
        for i in range(len(_bands) - 1):
            lower, upper = _bands[i], _bands[i+1]
            # add the suggested band overlap of 50%
            if i != 0:
                lower -= 0.5 * (_bands[i] - _bands[i-1])
            if i != len(_bands) - 2:
                upper += 0.5 * (_bands[i+2] - _bands[i+1])

            lower_bin, upper_bin = int(lower / freq_bin_size), int(upper / freq_bin_size)
            band_amplitudes = amplitudes[:, :, lower_bin:upper_bin]

            mean_amplitude_bands = func(band_amplitudes, axis=2)
            mean_amplitude_windows = np.mean(mean_amplitude_bands, axis=0)
            mean_amplitudes.extend(list(mean_amplitude_windows))

        return mean_amplitudes

In [ ]:
rec.raw_edf.load_data()
signals = rec.raw_edf.get_data()

data = pd.DataFrame(index=range(rec.n_samples), columns=rec.signal_names)
for electrode_id, electrode in enumerate(rec.signal_names):
    data[electrode] = signals[electrode_id]

In [ ]:
fnPrePAth = "hdf1/2/"

In [4]:
with h5py.File("1.hdf", "r") as f:
    a = f["data"]
    s = Segment(sampling_freq = a.attrs["sampling_freq"],
                n_samples = a.attrs["sampling_freq"] * a.attrs["duration"],
                duration = a.attrs["duration"],
                label = a.attrs["label"],
                signal_names = a.attrs["channel_names"],
               signals = a[:])

In [5]:
splitter = DataSplitter()

In [6]:
windows = splitter.split(s.signals, s.sampling_freq)
s.signals_ft = np.fft.rfft(windows, axis=2)
#s_ft = np.fft.rfft(windows, axis=2)
#s.signals_ft = np.abs(s_ft)

In [112]:
fft_extractors = sorted([feat_func for feat_func in dir(feature_frequency) if not feat_func.startswith('_')])

In [134]:
feature_labels = []

In [135]:
for freq_feat in fft_extractors:
    for band_id, band in enumerate(_bands[:-1]):
        for electrode in wanted_elecs:
            label = '_'.join(['fft', freq_feat, str(band) + '-' + str(_bands[band_id+1]) + 'Hz',
                              str(electrode)])
            feature_labels.append(label)
            
            

In [136]:
len(feature_labels)

1848

In [77]:
def rolling_to_windows(rolling_feature, window_size):
    """ This should be used to transform the results of the rolling operation of pandas to window values.
    beware! through ".diff" in pandas feature computation, a row of nan is inserted to the rolling feature.
    :param rolling_feature: feature computation achieved through pandas.rolling()
    :param window_size: number of samples in the window
    :return: rolling feature sliced at the window locations
    """
    overlap_size = int(0.5 * window_size)
    windowed_feature = rolling_feature[window_size-1::window_size-overlap_size]
    
    return windowed_feature



In [120]:
len(list(fft_extractors))

11

In [132]:
a = compute_freq_feats(fft_extractors[11], s, 2)
len(a)

IndexError: list index out of range

### len(a)

In [16]:
import numpy as _np

In [29]:
def _div0(x):
    x[x < 1e-9] = 1e-9
    return x

In [66]:
def fractaldim(df, window_size):

    sum_of_distances = _np.sqrt(df.diff() * df.diff()).rolling(window=window_size).sum()
    #sum_of_distances = pd.DataFrame(np.array(sum_of_distances)[window_size:,:])
    max_dist = df.rolling(window=window_size).apply(lambda df: _np.max(_np.sqrt(_np.square(df - df[0]))))

    return max_dist,_np.log10(sum_of_distances) / _div0(_np.log10(max_dist))


In [67]:
c = fractaldim(pd.DataFrame(s.signals.T), 500)

In [75]:
c[1][500:]

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
500,1.946717,1.966108,2.085656,2.115594,1.967820,2.059858,2.205227,2.226527,2.165190,2.059471,...,2.168033,2.266333,2.128805,2.032445,2.024931,1.926172,2.271931,2.211282,2.176241,2.054331
501,1.952336,1.942177,2.090803,2.067776,2.039835,1.941495,2.079649,2.066465,2.028372,2.029601,...,2.133256,2.180318,2.203940,2.011704,2.059967,2.002614,2.113511,2.094974,2.050633,2.096018
502,1.919655,1.913259,1.988613,2.109145,2.037890,1.849914,2.146307,1.969355,2.112897,1.921341,...,1.939164,2.140237,2.300255,1.925423,2.103296,2.006025,2.017414,2.185801,2.015865,2.057667
503,1.920775,1.924527,1.951016,1.953544,1.945659,1.813666,1.928326,1.981771,2.067957,1.840687,...,1.849208,2.110559,2.237019,1.912332,2.050852,1.988243,2.030706,2.119213,2.053087,2.154228
504,1.965545,1.950623,1.934293,1.968957,1.933431,1.826998,1.995031,1.960175,2.114978,1.878772,...,1.870834,2.109567,2.200056,1.886827,1.988161,1.958208,2.013145,2.137986,2.015544,2.217973
505,1.996286,2.009208,2.022741,2.093097,2.062693,1.932545,2.225328,2.036343,2.150044,2.006436,...,2.004921,2.185223,2.298900,1.935521,2.094877,2.010728,2.108964,1.981735,2.061888,2.210985
506,2.074381,2.077555,2.147786,2.085076,2.114573,2.035636,2.165620,2.282933,2.170466,2.075927,...,2.067927,2.277529,2.249704,2.045327,2.087296,2.028540,2.328606,1.977420,2.180037,2.169984
507,2.082576,2.097456,2.134665,1.939193,2.048060,2.038297,1.958144,2.326778,1.979823,2.011652,...,1.981406,2.258679,2.198799,2.019186,1.959237,1.994866,2.374600,2.126774,2.246944,2.132532
508,2.119096,2.094425,2.024429,2.072245,2.030427,2.084380,2.152339,2.182827,2.101356,2.092622,...,2.168542,2.216639,2.271194,1.992743,2.063508,2.003884,2.216726,2.189685,2.252108,2.128047
509,2.090193,2.094817,1.927270,1.988381,1.884700,1.963389,1.973840,2.163936,2.019997,1.949075,...,1.962603,2.181614,2.224365,1.920665,1.991777,1.874237,2.170046,2.017020,2.229982,2.246219


In [32]:
df = pd.DataFrame(s.signals)

In [8]:
import numpy as _np

In [33]:
a = []
for i in range(34):
     a.append(df[i*500:i*500+500] - df[0])
    

In [88]:
time_feats = sorted([feat_func for feat_func in dir(feature_time) if not feat_func.startswith('_')])

In [89]:
time_feats[2]

'fractaldim'

In [110]:
func = getattr(feature_time,time_feats[4])
# TODO: don't cast the signals here, originally safe them as pandas DataFrame
feature_values = func(pd.DataFrame(s.signals.T), 500)

feature_values = rolling_to_windows(feature_values[498:], 500)
feature_values = np.mean(feature_values, axis=0)

In [111]:
feature_values

0     1453.677634
1     1452.782807
2     2318.451732
3     4114.640406
4     2448.031055
5     2541.841875
6     4515.376371
7     2765.366083
8     3382.623547
9     2993.932749
10    2889.277758
11    2612.752212
12    1494.735399
13    1623.326209
14    1848.123105
15    2723.053431
16    2143.021891
17    3876.170578
18    2811.582408
19    1464.179450
20    1730.812555
dtype: float64

In [137]:
def compute_pyeeg_feats(rec):
        # these values are taken from the tuh paper
        TAU, DE, Kmax = 4, 10, 5
        pwrs, pwrrs, pfds, hfds, mblts, cmplxts, ses, svds, fis, hrsts = [], [], [], [], [], [], [], [], [], []
        dfas, apes = [], []

        for window_id, window in enumerate(rec.signals):
            for window_electrode_id, window_electrode in enumerate(window):
                # taken from pyeeg code / paper
                electrode_diff = list(np.diff(window_electrode))
                M = pyeeg.embed_seq(window_electrode, TAU, DE)
                W = scipy.linalg.svd(M, compute_uv=False)
                W /= sum(W)

                power, power_ratio = self.bin_power(window_electrode, self.bands, rec.sampling_freq)
                pwrs.extend(list(power))
                # mean of power ratio is 1/(len(self.bands)-1)
                pwrrs.extend(list(power_ratio))

                pfd = pyeeg.pfd(window_electrode, electrode_diff)
                pfds.append(pfd)

                hfd = pyeeg.hfd(window_electrode, Kmax=Kmax)
                hfds.append(hfd)

                mobility, complexity = pyeeg.hjorth(window_electrode, electrode_diff)
                mblts.append(mobility)
                cmplxts.append(complexity)

                se = self.spectral_entropy(window_electrode, self.bands, rec.sampling_freq, power_ratio)
                ses.append(se)

                svd = pyeeg.svd_entropy(window_electrode, TAU, DE, W=W)
                svds.append(svd)

                fi = pyeeg.fisher_info(window_electrode, TAU, DE, W=W)
                fis.append(fi)

                # this crashes...
                # ape = pyeeg.ap_entropy(electrode, M=10, R=0.3*np.std(electrode))
                # apes.append(ape)

                # takes very very long to compute
                # hurst = pyeeg.hurst(electrode)
                # hrsts.append(hurst)

                # takes very very long to compute
                # dfa = pyeeg.dfa(electrode)
                # dfas.append(dfa)

        pwrs = np.asarray(pwrs).reshape(rec.signals.shape[0], rec.signals.shape[1], len(self.bands)-1)
        pwrs = np.mean(pwrs, axis=0)

        pwrrs = np.asarray(pwrrs).reshape(rec.signals.shape[0], rec.signals.shape[1], len(self.bands)-1)
        pwrrs = np.mean(pwrrs, axis=0)

        pfds = np.asarray(pfds).reshape(rec.signals.shape[0], rec.signals.shape[1])
        pfds = np.mean(pfds, axis=0)

        hfds = np.asarray(hfds).reshape(rec.signals.shape[0], rec.signals.shape[1])
        hfds = np.mean(hfds, axis=0)

        mblts = np.asarray(mblts).reshape(rec.signals.shape[0], rec.signals.shape[1])
        mblts = np.mean(mblts, axis=0)

        cmplxts = np.asarray(cmplxts).reshape(rec.signals.shape[0], rec.signals.shape[1])
        cmplxts = np.mean(cmplxts, axis=0)

        ses = np.asarray(ses).reshape(rec.signals.shape[0], rec.signals.shape[1])
        ses = np.mean(ses, axis=0)

        svds = np.asarray(svds).reshape(rec.signals.shape[0], rec.signals.shape[1])
        svds = np.mean(svds, axis=0)

        fis = np.asarray(fis).reshape(rec.signals.shape[0], rec.signals.shape[1])
        fis = np.mean(fis, axis=0)

        return list(pwrs.ravel()), list(pwrrs.ravel()), pfds, hfds, mblts, cmplxts, ses, svds, fis, apes, hrsts, dfas

In [ ]:
compute_pyeeg_feats()